# 생성옵션 & 프롬프트 조정 작업

In [1]:
import pandas as pd
from datasets import Dataset
import json
import random
import numpy as np
import os
import torch
import pickle
from tqdm import tqdm
import time

## 1. 평가용 프롬프트 만들기

In [11]:
samples = pd.read_csv("gen_config_prompt_sample.csv", keep_default_na = False)
samples

###########
# cf. 태스크별 샘플 추출: gen_config_prompt_sample.csv 생성
# samples = []

# df_train = pd.read_csv("../../../node_storage2/data_llm_kr/data_it_train_240724.csv", keep_default_na = False)

# for task in df_train["task"].unique():
#     df_train_task = df_train[df_train["task"] == task ] # 특정 태스크에 해당하는 행 중에
#     sample = df_train_task.sample(n=1, random_state=1) # 1개 랜덤샘플링
#     samples.append(sample)

# samples = pd.concat(samples)
# samples.to_csv("gen_config_prompt_sample.csv", index=False)
###########

,task,instruction,input,output,option
0,kowiki_v2_leads,,황학 (명나라),황학(黃鶴)는 명나라 중기의 정치인이었다. 자(字)는 명고(鳴臯)이다. 하남성(河南...,
1,doosan_253717,,역추력장치,역추력장치는 제트엔진 등의 추력(推力)을 역전시켜서 바퀴의 브레이크 작용을 돕는 장...,
2,gen_generation,,여자친구와 5주년을 맞은 남자는 특별한 여행을 계획하고 있다. 남자는 인터넷으로 여...,앱이 화면에 나타나자 남자는 앱을 누른다.,
3,gen_closedQA_generation_3,명절을 기념해 지역 내 위기 청소년들에게 물품을 제공한 곳은 어디야,부산 서구 청소년상담복지센터(센터장 유은영) 청소년동반자들은 지난 8일부터 3일간 ...,부산 서구 청소년상담복지센터,
4,gen_closedQA_generation_2,맹형규 행안부 장관은 공직기강 확립을 위해 어떤 제목의 서한문을 전 직원들에게 발송했지,"행안부 긴급확대간부회의를 개최,「공직기강 확립 및 윤리 실천계획」추진키로 \n□ 맹...",행안부 가족에게 드리는 글,"['행안부 가족에게 드리는 글', '공직기강 확립 실천계획', '청렴서약서', '긴..."
5,gen_argument,,현재 기술자가 퇴사후에 직권으로 자격증 해임을 하기 위해서는 민법상 규정을 적용해 ...,퇴사시 기술자격증 해임처리방식을 바꾸어야 합니다.,
6,gen_summarization,,해양 녹색기술(green technology)개발은 재생가능하고 환경친화적 방식의 ...,해양 녹색기술개발은 재생가능하고 환경친화적 방식의 과학기술을 해양에서 적용하려는 시...,
7,translation,,This is most likely a result of cultural diffe...,이는 아마도 영국과 미국의 문화적 차이를 반영하는 듯하다.,
8,med_translation,,stress fracture,긴장골절 또는 피로골절,
9,med_definition,,울혈성심장기능상실,심장의 원래 기능 즉 혈액을 말초로 보내는 펌프와 같은 기능이 거의 없어진 ...,


In [3]:
# 원 데이터 태스크 전체 목록 
# len(df_train["task"].value_counts()), df_train["task"].value_counts()

(26,
 task
 doosan_253717                     228345
 kowiki_v2_leads                   180000
 gen_closedQA_generation_3          18000
 gen_assistant                      18000
 gen_closedQA_generation_2          13390
 oig-smallchip2-rvsd                12616
 med_translation                    12237
 gen_argument                       11871
 gen_generation                     11574
 gen_summarization                   9000
 law_summarization                   9000
 translation                         9000
 law_openQA                          3617
 med_definition                      2020
 law_statute_content                  982
 gen_closedQA_generation_1            900
 gen_closedQA_classification          900
 fin_definition                       669
 gen_news_classification              450
 gen_nli                              450
 gen_math                             450
 gen_sts_classification               450
 gen_polysemy_classification          450
 gen_sentiment_classifi

In [20]:
# pandas dataframe을 dataset 객체로 변환
ds_sample = Dataset.from_pandas(samples)
ds_sample

Dataset({
    features: ['task', 'instruction', 'input', 'output', 'option'],
    num_rows: 26
})

In [13]:
# 템플릿 파일 불러오기
with open('../../../node_storage2/data_llm_kr/instruction_template_240724.json', 'r') as f:
    templates = json.load(f)
templates

{'fin_definition': [['이 개념에 대해 설명해줘. \n\n{input}', '{output}'],
  ['{input}\n\n이건 무슨 뜻인가요?', '{output}'],
  ['이게 뭔지 간단하게 요약해줘.\n\n{input}', '{output}'],
  ['{input}\n\n이 단어가 의미하는 바가 무엇인지 짧게 설명해주세요.', '{output}'],
  ['다음 용어는 무슨 뜻인가요?\n\n{input}', '{output}'],
  ['다음 용어에 대한 정의를 알려주세요.\n\n{input}', '{output}'],
  ['이 단어 무슨 뜻이야? \n\n{input}', '{output}'],
  ['다음 단어가 뭔 뜻인지 알려줘.\n\n{input}', '{output}'],
  ['다음 단어가 어떤 의미인지 말해주십시오.\n\n{input}', '{output}'],
  ['이거 무슨 뜻이야?\n\n{input}', '{output}'],
  ['이게 뭐야?\n\n{input}', '{output}']],
 'gen_argument': [['다음 문장의 요지를 알려줘.\n\n{input}', '{output}'],
  ['아래 글은 무슨 내용의 글인지 정리해줘.\n\n{input}', '{output}'],
  ['{input}\n\n이것의 내용을 간략히 설명해주시겠어요?', '{output}'],
  ['{input}\n\n위 글의 핵심은?', '{output}'],
  ['이 문장이 무엇에 관한 내용인지 알려줘.\n\n{input}', '{output}'],
  ['다음과 같은 문장이 주어졌을 때 핵심 포인트를 알려줄래?\n\n{input}', '{output}'],
  ['다음 문장의 키워드를 알려주세요.\n\n{input}', '{output}'],
  ['이 문장에서 중요한 부분만 뽑아줘.\n\n{input}', '{output}'],
  ['다음 문장의 요지만 추려서 알려주세요.\n\n{input}', '{outp

In [18]:
# 데이터 포맷팅

# 1. 태스크별 질문 다르게 설정
def apply_task_templates(example):
    # 태스크별로 다른 질문 적용하기 위한 추가 함수.
    # 예시: input_text= "다음 내용{instruction}\n{option}\n\n{input}", output_text= "이 글의 카테고리는 {output} 입니다.""
    random_template = templates[example['task']][0] # 여러 템플릿 중 랜덤하게 선택하는 대신 (random.choice(templates[example['task']]) ) # 템플릿 중 0번째 항목 사용
    input_text = random_template[0]
    output_text = random_template[1]

    _instruction = example['instruction'].strip() if example['instruction'].strip() else ''
    _input = example['input'].strip() if example['input'].strip() else ''
    _output = example['output'].strip() if example['output'].strip() else ''
    _option = example['option'].strip() if example['option'].strip() else ''
    
    input_text = input_text.replace('{instruction}', _instruction)
    input_text = input_text.replace('{input}', _input)
    input_text = input_text.replace('{option}', _option)
    output_text = output_text.replace('{output}', _output)
        
    io_task_template_applied = (input_text, output_text)
    return io_task_template_applied
    

# 2. assistant 답변 생성용 템플릿 적용
  # 훈련 시에 사용하는 템플릿에서 output 부분 이후 제거
def formatting_func(examples):
    
    model2template_gen = {
        "meta-llama/Meta-Llama-3-8B-Instruct": """
        <|start_header_id|>system<|end_header_id|>
        당신의 역할은 한국어로 답변하는 **한국어 AI 어시트턴트**입니다. 주어진 질문에 대해 한국어로 답변해주세요.<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        아래 질문을 한국어로 정확하게 답변해주세요. **질문**: {}<|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>\n\n""", # post_processor adds '<|begin_of_text|>'
        "google/gemma-2-9b-it": """
        <start_of_turn>model
        당신의 역할은 한국어로 답변하는 **한국어 AI 어시트턴트**입니다. 주어진 질문에 대해 한국어로 답변해주세요.<end_of_turn>
        <start_of_turn>user
        아래 질문을 한국어로 정확하게 답변해주세요. **질문**: {}<end_of_turn>
        <start_of_turn>model\n""", # post_processor adds '<bos>'
        "Qwen/Qwen2-7B-Instruct": """
        <|im_start|>system
        당신의 역할은 한국어로 답변하는 **한국어 AI 어시트턴트**입니다. 주어진 질문에 대해 한국어로 답변해주세요.\n<|im_end|>
        <|im_start|>user
        아래 질문을 한국어로 정확하게 답변해주세요. **질문**: {}<|im_end|>
        <|im_start|>system\n"""
    }
    
    model2template_gen = {k:v.strip() for k, v in model2template_gen.items()}
    template = model2template_gen[model_name_or_path]
    
    formatted = []
    for i in range(len(examples["input"])):
        _input, _output = apply_task_templates(examples[i])
        formatted.append(template.format(_input)) # 훈련 시에는 template.format(_input, _output) 
        
    return formatted

In [21]:
ds_sample[0]

{'task': 'kowiki_v2_leads',
 'instruction': '',
 'input': '황학 (명나라)',
 'output': '황학(黃鶴)는 명나라 중기의 정치인이었다. 자(字)는 명고(鳴臯)이다. 하남성(河南省) 개봉부(開封府) 기현(杞縣) 출신이다.',
 'option': ''}

In [22]:
model_name_or_path = "meta-llama/Meta-Llama-3-8B-Instruct" # 어떤 모델 템플릿 적용할지   # "google/gemma-2-9b-it" # "Qwen/Qwen2-7B-Instruct"
ds_sample = formatting_func(ds_sample)

In [23]:
ds_sample

['<|start_header_id|>system<|end_header_id|>\n        당신의 역할은 한국어로 답변하는 **한국어 AI 어시트턴트**입니다. 주어진 질문에 대해 한국어로 답변해주세요.<|eot_id|>\n        <|start_header_id|>user<|end_header_id|>\n        아래 질문을 한국어로 정확하게 답변해주세요. **질문**: 이 개념에 대해 설명해줘. \n\n황학 (명나라)<|eot_id|>\n        <|start_header_id|>assistant<|end_header_id|>',
 '<|start_header_id|>system<|end_header_id|>\n        당신의 역할은 한국어로 답변하는 **한국어 AI 어시트턴트**입니다. 주어진 질문에 대해 한국어로 답변해주세요.<|eot_id|>\n        <|start_header_id|>user<|end_header_id|>\n        아래 질문을 한국어로 정확하게 답변해주세요. **질문**: 이 개념에 대해 설명해줘. \n\n역추력장치<|eot_id|>\n        <|start_header_id|>assistant<|end_header_id|>',
 '<|start_header_id|>system<|end_header_id|>\n        당신의 역할은 한국어로 답변하는 **한국어 AI 어시트턴트**입니다. 주어진 질문에 대해 한국어로 답변해주세요.<|eot_id|>\n        <|start_header_id|>user<|end_header_id|>\n        아래 질문을 한국어로 정확하게 답변해주세요. **질문**: 이 글 다음에 올 문장을 알려줘.\n\n여자친구와 5주년을 맞은 남자는 특별한 여행을 계획하고 있다. 남자는 인터넷으로 여행 장소를 검색하고 정보를 수집하기 시작한다. 그리고 정보를 찾다가 알아낸 여행 관련 앱을 태블릿에 다운받는다.<|eot_id|>\n        <|star

In [30]:
# 프롬프트 파일 저장하려면
with open('gen_config_prompt.pickle', "wb") as f:
    pickle.dump(ds_sample, file=f)

## 2. 평가용 프롬프트 불러오기

In [34]:
# 저장한 프롬프트 파일 불러오려면
with open('gen_config_prompt.pickle', "rb") as f:
    ds_sample = pickle.load(f)

In [24]:
ds_sample[0]

'<|start_header_id|>system<|end_header_id|>\n        당신의 역할은 한국어로 답변하는 **한국어 AI 어시트턴트**입니다. 주어진 질문에 대해 한국어로 답변해주세요.<|eot_id|>\n        <|start_header_id|>user<|end_header_id|>\n        아래 질문을 한국어로 정확하게 답변해주세요. **질문**: 이 개념에 대해 설명해줘. \n\n황학 (명나라)<|eot_id|>\n        <|start_header_id|>assistant<|end_header_id|>'

cf. 훈련 시 사용한 템플릿 적용 결과
- assistant 뒤에 데이터의 output 칼럼 값 들어감
```
"""'<|start_header_id|>system<|end_header_id|>\n        당신의 역할은 한국어로 답변하는 **한국어 AI 어시트턴트**입니다. 주어진 질문에 대해 한국어로 답변해주세요.<|eot_id|>\n        <|start_header_id|>user<|end_header_id|>\n        아래 질문을 한국어로 정확하게 답변해주세요. **질문**: 다음 단어를 정의 중심으로 설명해주세요.\n\n황학 (명나라)<|eot_id|>\n        <|start_header_id|>assistant<|end_header_id|>\n\n황학(黃鶴)는 명나라 중기의 정치인이었다. 자(字)는 명고(鳴臯)이다. 하남성(河南省) 개봉부(開封府) 기현(杞縣) 출신이다.<|eot_id|>\n        <|end_of_text|>'"""
```

## 3. 프롬프트 & 생성옵션 조정하기

In [25]:
def set_seed(seed):    
    # -> config.py에서 설정한 시드가 모델훈련 등 다른 모듈에도 적용되는지 체크 필요
    
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  


def load_model(model_path, peft):
    if peft: # lora 훈련 모델
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        
        config = PeftConfig.from_pretrained(model_path) # 훈련시킨 어댑터 로드 # LoraConfig
        model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
        model.resize_token_embeddings(len(tokenizer))
        
        model = PeftModelForCausalLM.from_pretrained(model, model_path)

    else: # default 훈련 모델
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.bfloat16, 
            low_cpu_mem_usage=True,
            device_map="auto")

    setattr(model, 'model_parallel', True)
    setattr(model, 'is_parallelizable', True)
    model.config.use_cache = False 
    model.config.pretraining_tp = 1
    model.config.pad_token_id = model.config.eos_token_id
    
    return model


def load_tokenizer(model_path):
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    
    return tokenizer

In [26]:
# 기본 설정
from transformers import AutoModelForCausalLM, AutoTokenizer

set_seed(seed=1) # 시드 고정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 불러오기
# 모델 로드 안되는 경우 : 터미널에서 huggingface-cli login 실행 후 access token 입력
model = load_model(model_name_or_path, peft=False)
tokenizer = load_tokenizer(model_name_or_path)
model.resize_token_embeddings(len(tokenizer))
print("len(tokenizer):", len(tokenizer))    

/home/hmcho/.conda/envs/model_train_py39/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


len(tokenizer): 128256


In [27]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [24]:
# TODO: 값 조정하기
gen_config = {'max_new_tokens': 256, 'do_sample': True, 'temperature': 0.8, 'top_p': 1.0, 'top_k': 5, 'no_repeat_ngram_size': 0}

* 참고사항

1. 정성평가 코드에서 설정된 기본값 (24-07-25 기준)
```
 'generation config': {'max_new_tokens': 256, 'do_sample': True, 'temperature': 1.0, 'top_p': 1.0, 'top_k': 5, 'no_repeat_ngram_size': 0}
```

2. 정성평가에서 현재 사용하는 hyperparameter 값
```
 'generation config': {'max_new_tokens': 256, 'do_sample': True, 'temperature': 0.8, 'top_p': 1.0, 'top_k': 5, 'no_repeat_ngram_size': 0}
```

In [25]:
# 각 프롬프트에 대해 output 생성시키기
def get_gen_results(gen_config, prompts):
    responses = []
    batched_inputs = tokenizer(prompts, padding=True, return_tensors="pt", return_token_type_ids=False).to(device)

    with torch.no_grad():
        start_time = time.time()
        outputs = model.generate(**batched_inputs, **gen_config)
        print(f"=========================== total generation took {time.time()-start_time:.2f} sec. ===========================")
        response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        responses.append(response)
        
    return responses  

In [26]:
# 생성 테스트 (-> colab A100 GPU, llama3 기준으로 총 17초 소요) 
result = get_gen_results(gen_config, ds_sample)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KeyboardInterrupt: 

In [ ]:
for item in result:
    print(item)
    print()